In [ ]:
from pathlib import Path
import sys

lib_path = str(Path('../').resolve())
sys.path.insert(0, lib_path)
lib_path

In [ ]:
from crops import CropModels


csv_fp = Path('../../../assets') / 'sample.csv'

crop_models = {ModelClass.key: ModelClass for ModelClass in CropModels}
for crop_model in crop_models.values():
    crop_model = crop_model()
    print(crop_model.name, ': ', crop_model.key)

### Events, Schedules 프로파일러

In [ ]:
from datetime import datetime

import pandas as pd


def get_human_time(doy):
  year = 2022
  ts = datetime(year, 1, 1).timestamp() + (doy - 1) * 24 * 3600
  dt = datetime.fromtimestamp(ts)
  return f'{dt.month}월 {dt.day}일'


def get_event_start_doy(crop_model, param, method):
    if method == 'GDD':
        event = crop_model.calculate_gdd_hyperparam(param)
    elif method == 'DOY':
        ref = crop_model.calculate_first_priority_params()
        event = crop_model.calculate_doy_hyperparam(param, ref)
    doys = event['data'] 
    start_doy = doys[0] if isinstance(doys, list) else doys
    return start_doy


def get_event_date(crop_model, param, method):
    if method == 'GDD':
        event = crop_model.calculate_gdd_hyperparam(param)
    elif method == 'DOY':
        ref = crop_model.calculate_first_priority_params()
        event = crop_model.calculate_doy_hyperparam(param, ref)
    doys = event['data'] 
    if isinstance(doys, list):
            date = ' ~ '.join([get_human_time(doy) for doy in doys])
    else:
        date = get_human_time(doys)
    return date


def interpret_rule(crop_model, param):
    value = param['value']
    ref = param.get('ref', [])

    # 계산 기준시기가 있을 경우: 현재로서는 모두 doy 기반 계산임
    method_type = 'DOY' if len(ref) > 0 else 'GDD'
    if method_type == 'DOY':  
        all_events = crop_model.gdd_hyperparams + crop_model.doy_hyperparams
        ret = []
        for _ref, _idx, _val in zip(ref, param.get('index', [None]*len(ref)), value):
            filtered_ret = filter(lambda x: x['type'] == _ref, all_events)
            
            _prefix = next(filtered_ret)
            prefix = _prefix['name']
            suffix = ''
            if _idx == 0: suffix = '시작'
            if _idx == 1: suffix = '종료'

            if _val == 0:
                ret.append(f'{prefix}')
            else:
                postfix = f'전' if _val < 0 else '후'
                ret.append(f'{prefix} {suffix} {abs(_val)}일 {postfix}')
        return ' ~ '.join(ret)

    elif method_type == 'GDD':
        if isinstance(value, list):
            return f'{value[0]}~{value[1]}'
        else:
            ret = str(value[0]) if isinstance(value, list) else str(value)
            return ret


def profile_all_events_and_schedules(crop_model):
    ref_events = crop_model.calculate_first_priority_params()
     
    # gdd 기반 계산이 필요한 event들 계산
    method = 'GDD'
    for param in crop_model.gdd_hyperparams:
        event = crop_model.calculate_gdd_hyperparam(param)
        
        start_doy = get_event_start_doy(crop_model, param, method)
        date = get_event_date(crop_model, param, method)
        rule = interpret_rule(crop_model, param)

        ret = {
          **event, 
          'rule': rule,
          'date': date,
          'max_period': str(param.get('max_period', '')),
          'event_start_doy': start_doy
        }
        yield ret

    # doy 기반 계산이 필요한 event들 계산
    method = 'DOY'
    for param in crop_model.doy_hyperparams:
        event = crop_model.calculate_doy_hyperparam(param, ref_events)
        start_doy = get_event_start_doy(crop_model, param, method)
        date = get_event_date(crop_model, param, method)
        rule = interpret_rule(crop_model, param)
        
        ret = {
          **event, 
          'rule': rule,
          'date': date,
          'max_period': str(param.get('max_period', '')),
          'event_start_doy': start_doy
        }
        yield ret



### Warnings 프로파일러

In [ ]:
OP_STR = {
  'ge': '이상',
  'gt': '초과',
  'e': '동일',
  'le': '이하',
  'lt': '미안',
}
VAR_STR = {
  'tmax': '최고기온',
  'tmin': '최저기온',
  'tavg': '평균기온',
}

def profile_warnings(crop_model):
    ref_events = crop_model.calculate_first_priority_params()

    ret = {}
    for param in crop_model.warning_hyperparams:
        method = param['method']

        if method == 'temperature_and_exposure':
            _ret = {
                '고온해': f'''{
                    param['high_extrema_temperature']
                  }℃ 이상(초과) {
                    param['high_extrema_exposure_days']
                  }일 이상 노출''',
                '동해': f'''{
                    param['low_extrema_temperature']
                  }℃ 이하(미만) {
                    param['low_extrema_exposure_days']
                  }일 이상 노출'''
            }
            ret.update(_ret)

        elif method == 'milestone_and_temperature_condition':
            all_events = crop_model.gdd_hyperparams + crop_model.doy_hyperparams

            ref = param['milestone']['ref']
            value = param['milestone']['value']

            milestone_rule = []
            for _ref, _idx, _val in zip(ref, param.get('index', [None]*len(ref)), value):
                filtered_ret = filter(lambda x: x['type'] == _ref, all_events)
                
                _prefix = next(filtered_ret)
                prefix = _prefix['name']
                suffix = ''
                if _idx == 0: suffix = '시작'
                if _idx == 1: suffix = '종료'

                if _val == 0:
                    milestone_rule.append(f'{prefix}')
                else:
                    postfix = f'전' if _val < 0 else '후'
                    milestone_rule.append(f'{prefix} {suffix} {abs(_val)}일 {postfix}')
            milestone_str = ' ~ '.join(milestone_rule)
            
            cond = param['condition']
            variable = cond['variable']
            temperature = cond['temperature']
            operator = cond['operator']

            warning_title = param['warning_data']['title']
            ret.update({warning_title: f'{milestone_str} 때 {VAR_STR[variable]} {temperature}℃ {OP_STR[operator]}'})
            _ret = {}
            ret.update(_ret)

        elif method == 'milestone_length_condition':
            all_events = crop_model.gdd_hyperparams + crop_model.doy_hyperparams

            ref = param['milestone']['ref']
            value = param['milestone']['value']

            milestone_rule = []
            for _ref, _idx, _val in zip(ref, param.get('index', [None]*len(ref)), value):
                filtered_ret = filter(lambda x: x['type'] == _ref, all_events)
                
                _prefix = next(filtered_ret)
                prefix = _prefix['name']
                suffix = ''
                if _idx == 0: suffix = '시작'
                if _idx == 1: suffix = '종료'

                if _val == 0:
                    milestone_rule.append(f'{prefix}')
                else:
                    postfix = f'전' if _val < 0 else '후'
                    milestone_rule.append(f'{prefix} {suffix} {abs(_val)}일 {postfix}')
            milestone_str = ' ~ '.join(milestone_rule)
            
            cond = param['condition']
            length = cond['length']
            operator = cond['operator']

            warning_title = param['warning_data']['title']
            ret.update({warning_title: f'{milestone_str}이 {length}일 {OP_STR[operator]}'})
    return ret

In [ ]:
from utils.weather import get_weather_df


crop_key = 'onion'
crop_model = crop_models.get(crop_key)()
weather_df = get_weather_df(csv_fp)
crop_model.set_weather_data(weather_df)

meta = {
  '작물명': crop_model.name,
  'GDD 계산방법': crop_model.gdd_method,
  'GDD 기준온도': crop_model.base_temperature,
  'GDD 최대생육온도': crop_model.max_dev_temperature,
}

items = ({
      '일정': event['name'],
      '날짜': event['date'],
      'GDD': event['rule'],
      '최대일수': event.get('max_period', ''),
      'evt_st': event['event_start_doy'],
      '비고': event.get('text', '')
    } for event in profile_all_events_and_schedules(crop_model))

prof_df = pd.DataFrame\
          .from_records(items)\
          .sort_values('evt_st').drop('evt_st', axis=1)

for k, v in meta.items():
    print(f'{k}: {v}')
display(prof_df)
for k, v in profile_warnings(crop_model).items():
    print(f'{k}: {v}')

    

### RUN ALL

In [ ]:
for crop_key in crop_models:
    crop_model = crop_models.get(crop_key)()
    weather_df = get_weather_df(csv_fp)
    crop_model.set_weather_data(weather_df)

    meta = {
      '작물명': crop_model.name,
      'GDD 계산방법': crop_model.gdd_method,
      'GDD 기준온도': crop_model.base_temperature,
      'GDD 최대생육온도': crop_model.max_dev_temperature,
    }

    items = ({
          '일정': event['name'],
          '날짜': event['date'],
          'GDD': event['rule'],
          '최대일수': event.get('max_period', ''),
          'evt_st': event['event_start_doy'],
          '비고': event.get('text', '')
        } for event in profile_all_events_and_schedules(crop_model))

    prof_df = pd.DataFrame\
              .from_records(items)\
              .sort_values('evt_st').drop('evt_st', axis=1)

    for k, v in meta.items():
        print(f'{k}: {v}')
    display(prof_df)
    # for k, v in profile_warnings(crop_model).items():
    #     print(f'{k}: {v}')